In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J'

In [3]:
directory_in = './file/3.make'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [4]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Part Code'] = df_input['Part Code'].astype(int)
df_input['Make Code'] = df_input['Make Code'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：5315



,Part Code,Make,Make Code,JOIN_PCMC,Url
0,13362498,LEXUS,842,13362498;842,https://web.tecalliance.net/tecdoc/qa/parts/21...
1,13362498,TOYOTA,111,13362498;111,https://web.tecalliance.net/tecdoc/qa/parts/21...
2,13362499,DACIA,139,13362499;139,https://web.tecalliance.net/tecdoc/qa/parts/21...
3,13362499,MITSUBISHI,77,13362499;77,https://web.tecalliance.net/tecdoc/qa/parts/21...
4,13362499,RENAULT,93,13362499;93,https://web.tecalliance.net/tecdoc/qa/parts/21...
...,...,...,...,...,...
5310,1396752197,MERCEDES-BENZ,74,1396752197;74,https://web.tecalliance.net/tecdoc/qa/parts/21...
5311,1396752267,MERCEDES-BENZ,74,1396752267;74,https://web.tecalliance.net/tecdoc/qa/parts/21...
5312,1396752376,MERCEDES-BENZ,74,1396752376;74,https://web.tecalliance.net/tecdoc/qa/parts/21...
5313,1396752464,MERCEDES-BENZ,74,1396752464;74,https://web.tecalliance.net/tecdoc/qa/parts/21...


In [5]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Part Code                                             13362498
 Make                                                     LEXUS
 Make Code                                                  842
 JOIN_PCMC                                         13362498;842
 Url          https://web.tecalliance.net/tecdoc/qa/parts/21...
 Name: 0, dtype: object)

In [6]:
import time

data = {'getArticleLinkedAllLinkingTarget4': {'articleCountry': 'DE',
                                              'articleId': crawler_series['Part Code'],
                                              'country': 'DE',
                                              'countryGroupFlag': False,
                                              'lang': 'qa',
                                              'linkingTargetManuId': crawler_series['Make Code'],
                                              'linkingTargetType': 'VOLB',
                                              'provider': 23365,
                                              'withMainArticles': False}}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'x-api-key': api}),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
            elif dict_['status'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp

1


<Response [200]>

In [7]:
dict_ = resp.json()

dict_

{'data': {'array': [{'articleLinkages': {'array': [{'articleLinkId': 994698707,
       'linked': True,
       'linkingTargetId': 53026,
       'linkingTargetType': 'P'}]}}]},
 'status': 200}

In [8]:
dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed'

False

In [9]:
list_link = dict_['data']['array'][0]['articleLinkages']['array']

list_link

[{'articleLinkId': 994698707,
  'linked': True,
  'linkingTargetId': 53026,
  'linkingTargetType': 'P'}]

In [10]:
list_link

[{'articleLinkId': 994698707,
  'linked': True,
  'linkingTargetId': 53026,
  'linkingTargetType': 'P'}]

In [11]:
list_dict_link, index_begin, index_end = [], 0, 25
while True:
    dict_link = {'array': []}
    if index_end < len(list_link):
        for link in list_link[index_begin:index_end]:
            dict_link['array'].append({'articleLinkId': link['articleLinkId'], 'linkingTargetId': link['linkingTargetId']})
        list_dict_link.append(dict_link)
        index_begin, index_end = index_end, index_end + 25
    else:
        for link in list_link[index_begin:]:
            dict_link['array'].append({'articleLinkId': link['articleLinkId'], 'linkingTargetId': link['linkingTargetId']})
        list_dict_link.append(dict_link)
        break

list_dict_link

[{'array': [{'articleLinkId': 994698707, 'linkingTargetId': 53026}]}]

In [12]:
list_dict_correct.extend([{'Part Code': crawler_series['Part Code'],
                           'Make': crawler_series['Make'],
                           'Make Code': crawler_series['Make Code'],
                           'JOIN_PCMC': crawler_series['JOIN_PCMC'],
                           'Url': crawler_series['Url'],
                           'Link No': i + 1,
                           'Json_Link': json.dumps(dict_link),
                           'JOIN_PCMCLN': f'''{crawler_series['JOIN_PCMC']};{i + 1}'''} for i, dict_link in enumerate(list_dict_link)])

list_dict_correct

[{'Part Code': 13362498,
  'Make': 'LEXUS',
  'Make Code': 842,
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/574085/detail?groups=298',
  'JOIN_PCMC': '13362498;842',
  'Link No': 1,
  'Json_Link': '{"array": [{"articleLinkId": 994698707, "linkingTargetId": 53026}]}',
  'JOIN_JOIN_PCMCLN': '13362498;842;1'}]

In [13]:
crawler_status = True

crawler_status

True

In [14]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Part Code                                             13362498
 Make                                                     LEXUS
 Make Code                                                  842
 JOIN_PCMC                                         13362498;842
 Url          https://web.tecalliance.net/tecdoc/qa/parts/21...
 Check_Url    https://web.tecalliance.net/tecdoc/qa/parts/21...
 Name: 0, dtype: object]

In [15]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make', 'Link No'],
                                                             ascending=[True, True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_link.xlsx', index=False)

df_correct

,Part Code,Make,Make Code,Url,JOIN_PCMC,Link No,Json_Link,JOIN_JOIN_PCMCLN
0,13362498,LEXUS,842,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362498;842,1,"{""array"": [{""articleLinkId"": 994698707, ""linki...",13362498;842;1


In [16]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Code', 'Make'],
                                                           ascending=[True, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_link_error.xlsx', index=False)

df_error

,Part Code,Make,Make Code,JOIN_PCMC,Url,Check_Url
0,13362498,LEXUS,842,13362498;842,https://web.tecalliance.net/tecdoc/qa/parts/21...,https://web.tecalliance.net/tecdoc/qa/parts/21...
